In [8]:
# Original expression
original = lambda old_value, new_value: (
    (old_value == 0) or
    (old_value > 0 and (old_value + new_value < 0 or old_value + new_value > old_value)) or
    (old_value < 0 and (old_value + new_value > 0 or old_value + new_value < old_value))
)

# Optimized expression
optimized = lambda old_value, new_value: (
    (old_value != 0 and (old_value * (old_value + new_value) < 0 or (new_value > 0 and old_value > 0) or (new_value < 0 and old_value < 0))) 
    or (old_value == 0)
)

# Test function
def test_equivalence(func1, func2):
    test_cases = [
        (0, 10), (0, -10), (20, -40), (20, -30), (20, -20), (20, -10), (20, 10),
        (-20, 40), (-20, 30), (-20, 20), (-20, 10), (-20, -10)
    ]
    for old, new in test_cases:
        assert func1(old, new) == func2(old, new), f"Mismatch for {old}, {new}"
    print("All test cases passed. The functions are equivalent.")

# Run the test
test_equivalence(original, optimized)


All test cases passed. The functions are equivalent.
